In [12]:
import os
os.chdir('..')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import scipy.stats as stats
import sparklyRGT as rgt
pd.options.mode.chained_assignment = None
pd.set_option('display.max_rows',100)

In [108]:
os.getcwd()
os.chdir('C:\\Users\\dexte\\sparklyRGT\\data')
file_names = ['BH06_raw_round1-infusions.xlsx', 'BH06_raw_round1-makeup.xlsx'] 
df = rgt.load_data(file_names)
# rgt.check_sessions(df)

In [109]:
df2 = rgt.edit_groups(df, orig_group = [0], new_group = [3], subs = [5])
df2 = rgt.drop_subjects(df, subs = [7])
df2 = rgt.get_choices(df)
os.chdir('C:\\Users\\dexte\\sparklyRGT\\data')
file_names = ['BH06_raw_round1-infusions.xlsx


In [132]:
##will have to write it so that session or group can be passed 
## right now it's going by Group

def prems_after_outcome(df, sessions = 'All'):
    if sessions == 'All':
        sess = df.Group.unique() #all unique sessions
        sess.sort()
    subs = df.Subject.unique() #all unique subjects
    subs.sort()
    df2 = pd.DataFrame(columns = ['Subject','Session','Prems_after_win','Prems_after_loss']) #creates 4 columns
    for sub in subs:
        for num in sess:
            df1 = df.loc[(df.Subject == sub) & (df.Group == num)] #df1 is the raw df for a certain subject and session
            if df1.empty: #if df1 is empty
                df_temp = pd.DataFrame(data = {'Subject': sub, 'Session': num, 'Prems_after_win': np.nan,
                                               'Prems_after_loss': np.nan}, index = [0]) #make one row with NaN 
                df2 = df2.append(df_temp, ignore_index = True) #append that row to df2 
            else:
                prems_after_win = 0
                prems_after_loss = 0
                for i in range(1,len(df1)): #for 2nd trial to rest of trials
                    prev_trial = df1.iloc[i-1, :] #previous trial
                    curr_trial = df1.iloc[i, :] #current trial
                    if curr_trial.Premature_Resp == 1: #if a premature choice occurs...
                        if prev_trial.Rewarded == 1: #if previous trial was a win
                            prems_after_win += 1 #add 1 to prems_after_win
                        if prev_trial.Rewarded == 0 and prev_trial.Premature_Resp == 0: #if previous trial was a loss (and not a premature response)
                            prems_after_loss += 1 #add 1 to prems_after_loss
                df_temp = pd.DataFrame(data = {'Subject': sub, 'Session': num, 'Prems_after_win': prems_after_win,
                                               'Prems_after_loss': prems_after_loss}, index = [0]) #make the row for this subject and session
                df2 = df2.append(df_temp, ignore_index = True) #append this row to df2 
    return df2

pd.set_option('display.max_rows', None)
prem_df = prems_after_outcome(df2) #NaNs are occurring due to df1.empty! could remove 

In [133]:


for i in range(1,5):
    df3 = prem_df.loc[prem_df['Session']==i]
    print(df3.Prems_after_win.sum(), df3.Prems_after_loss.sum())


143 77
153 80
134 103
154 60


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

In [91]:
sess = df.Session.unique()
subs = df.Subject.unique()

df2 = pd.DataFrame(columns = ['Subject','Session','Prems_after_win','Prems_after_loss'])
df1 = df.loc[(df.Subject == 16) & (df.Session == 78)]
df_temp = pd.DataFrame(data = {'Subject': 16, 'Session': 78, 'Prems_after_win': np.nan,
                                               'Prems_after_loss': np.nan}, index = [0])
df2 = df2.append(df_temp, ignore_index = True)
df_temp = pd.DataFrame(data = {'Subject': 16, 'Session': 76, 'Prems_after_win': np.nan,
                                               'Prems_after_loss': np.nan}, index = [0])
df2 = df2.append(df_temp, ignore_index = True)
df1 = df.loc[(df.Subject == 16) & (df.Session == 55)]
range(1,len(df1))
prev_trial = df1.iloc[14-1, :]
prev_trial.Rewarded
curr_trial = df1.iloc[14, :]
curr_trial.Premature_Resp


1

## Copied code

In [ ]:
#prems_after_win still split by risky_choice 
# def prems_after_win_split(df, sessions = 'All'):
#     if sessions == 'All':
#         sess = df.Session.unique() #all unique sessions
#     subs = df.Subject.unique() #all unique subjects
#     df2 = pd.DataFrame(columns = ['Subject','Session','Win-stay_risk','Win-stay_optimal']) #creates 4 columns
#     for sub in subs:
#         for num in sess:
#             df1 = df.loc[(df.Subject == sub) & (df.Session == num)] #df1 is the raw df for a certain subject and session
#             if df1.empty: #if df1 is empty
#                 df_temp = pd.DataFrame(data = {'Subject': sub, 'Session': num, 'Win-stay_risk': np.nan,
#                                            'Win-stay_optimal': np.nan}, index = [0]) #make one row with NaN 
#                 df2 = df2.append(df_temp, ignore_index = True) #append that row to df2 
#             else:
#                 risky_win_stay = 0 #stay after risky_win
#                 optimal_win_stay = 0
#                 for i in range(1,len(df1)): #for 2nd trial to rest of trials
#                     prev_trial = df1.iloc[i-1, :] #previous trial
#                     curr_trial = df1.iloc[i, :] #current trial
#                     if curr_trial.risky_choice == 1: #if a risky choice occurs...
#                         if prev_trial.risky_choice == 1 and prev_trial.Rewarded == 1: #if previous trial was a risky choice, and a win
#                             risky_win_stay += 1 #add 1 to risky_win_stay
#                     elif curr_trial.risky_choice == 0: #if an optimal choice occurs 
#                         if prev_trial.risky_choice == 0 and prev_trial.Rewarded == 1: #if previous trial was an optimal choice, and a win
#                             optimal_win_stay += 1 #add 1 to optimal_win_stay
#                 df_temp = pd.DataFrame(data = {'Subject': sub, 'Session': num, 'Win-stay_risk': risky_win_stay,
#                                                'Win-stay_optimal': optimal_win_stay}, index = [0]) #make the row for this subject and session
#                 df2 = df2.append(df_temp, ignore_index = True) #append this row to df2 
#     return df2

In [48]:
def get_win_stay(df, sessions = 'All'):
    if sessions == 'All':
        sess = df.Group.unique()
    subs = df.Subject.unique()
    df2 = pd.DataFrame(columns = ['Subject','Session','Win-stay_risk','Win-stay_optimal'] )
    for sub in subs:
        for num in sess:
            df1 = df.loc[(df.Subject == sub) & (df.Group == num)]
            if df1.empty:
                df_temp = pd.DataFrame(data = {'Subject': sub, 'Session': num, 'Win-stay_risk': np.nan,
                                           'Win-stay_optimal': np.nan}, index = [0])
                df2 = df2.append(df_temp, ignore_index = True)
            else:
                risky_win_stay = 0
                optimal_win_stay = 0
                for i in range(1,len(df1)):
                    prev_trial = df1.iloc[i-1, :]
                    curr_trial = df1.iloc[i, :]
                    if curr_trial.risky_choice == 1:
                        if prev_trial.risky_choice == 1 and prev_trial.Rewarded == 1:
                            risky_win_stay += 1 
                    elif curr_trial.risky_choice == 0:
                        if prev_trial.risky_choice == 0 and prev_trial.Rewarded == 1:
                            optimal_win_stay += 1
                df_temp = pd.DataFrame(data = {'Subject': sub, 'Session': num, 'Win-stay_risk': risky_win_stay,
                                               'Win-stay_optimal': optimal_win_stay}, index = [0])
                df2 = df2.append(df_temp, ignore_index = True)
    return df2

def get_lose_shift(df, subs, sess):
    df2 = pd.DataFrame(columns = ['Lose-shift_risk','Lose-shift_optimal'] )
    for sub in subs:
        for num in sess:
            df1 = df.loc[(df.Subject == sub) & (df.Group == num)]
            if df1.empty:
                df_temp = pd.DataFrame(data = {'Lose-shift_risk': np.nan,
                                           'Lose-shift_optimal': np.nan}, index = [0])
                df2 = df2.append(df_temp, ignore_index = True)
            else:
                risky_lose_shift = 0
                optimal_lose_shift = 0
                for i in range(1,len(df1)):
                    prev_trial = df1.iloc[i-1, :]
                    curr_trial = df1.iloc[i, :]
                    if curr_trial.risky_choice == 0:
                        if prev_trial.risky_choice == 1 and prev_trial.Rewarded == 0:
                            risky_lose_shift += 1 
                    elif curr_trial.risky_choice == 1:
                        if prev_trial.risky_choice == 0 and prev_trial.Rewarded == 0:
                            optimal_lose_shift += 1
                df_temp = pd.DataFrame(data = {'Lose-shift_risk': risky_lose_shift,
                                               'Lose-shift_optimal': optimal_lose_shift}, index = [0])
                df2 = df2.append(df_temp, ignore_index = True)
    return df2

In [24]:
sub = df.Subject.unique()
sess = df.Group.unique()
df3 = get_win_stay(df2, sub, sess)
df4 = get_lose_shift(df2, sub, sess)

TypeError: get_win_stay() takes from 1 to 2 positional arguments but 3 were given

In [49]:
# sub = df.Subject.unique()
# sess = df.Group.unique()
df3 = get_win_stay(df2)
df3
# df4 = get_lose_shift(df2)

,Subject,Session,Win-stay_risk,Win-stay_optimal
0,9,1,32,0
1,9,4,22,0
2,9,3,22,1
3,9,2,24,1
4,11,1,25,0
...,...,...,...,...
51,6,2,0,32
52,8,1,15,8
53,8,4,12,14
54,8,3,11,2


In [6]:
df5 = pd.concat([df3,df4], axis = 1)
df5.sort_values(by = ['Subject','Session'], inplace = True, ignore_index = True)
df5

,Subject,Session,Win-stay_risk,Win-stay_optimal,Lose-shift_risk,Lose-shift_optimal
0,1,1,15,7,7,3
1,1,2,8,23,13,6
2,1,3,2,6,3,1
3,1,4,1,63,6,8
4,2,1,6,22,14,2
5,2,2,1,31,11,3
6,2,3,10,14,16,3
7,2,4,2,46,9,2
8,3,1,19,1,5,1
9,3,2,12,1,2,1


In [12]:
trials = df2.groupby(['Subject', 'Group'],as_index=False)['Trial'].max()
trials.loc[:,'Trial']

0      50.0
1      79.0
2      40.0
3     111.0
4      77.0
5      74.0
6      70.0
7      94.0
8      50.0
9      37.0
10     45.0
11     40.0
12     86.0
13     82.0
14     95.0
15     79.0
16     78.0
17     79.0
18     87.0
19    128.0
20    124.0
21     98.0
22    105.0
23    121.0
24     75.0
25     58.0
26     56.0
27     66.0
28     64.0
29     69.0
30     55.0
31     48.0
32     55.0
33     58.0
34     61.0
35     55.0
36     63.0
37     80.0
38     95.0
39     55.0
40     63.0
41     69.0
42     63.0
43     87.0
44     88.0
45     90.0
46     86.0
47     55.0
48     59.0
49     67.0
50     43.0
51    103.0
52     95.0
53    107.0
54     97.0
Name: Trial, dtype: float64